In [5]:
import numpy as np
import time
from utils import model
from inference import log_marginal_likelihood
from inference import zstates as zs
from inference import zstates_old_method as zs_old
from utils import hyperparameters

pi = 0.1
mu = 0.0
sigmabg = 0.001
sigma = 0.3
tau = 1 / (0.05 * 0.05)

x, y, csnps, v = model.simulate(pi = pi,
                             mu = mu,
                             sigma = sigma,
                             sigmabg = sigmabg,
                             tau = tau)

nvar = x.shape[0]
nsample = x.shape[1]
params = np.array([pi, mu, sigma, sigmabg, tau])
cmax = 1
scaledparams = hyperparameters.scale(params)
zstates_new = zs.create(scaledparams, x, y, cmax, nvar, 0.98)
zstates_old = zs_old.create(scaledparams, x, y, cmax, nvar, 0.98)

In [9]:
from scipy import optimize
init_params = np.array([0.005, 0.0, 0.1, 0.1, 1 / 0.5 / 0.5])
scaledparams = hyperparameters.scale(init_params)
bounds = [[scaledparams[i], scaledparams[i]] for i in range(5)]
bounds[0] = [None, None]
#bounds[1] = [None, None]
bounds[2] = [None, None]
bounds[3] = [None, None]
bounds[4] = [None, None]
args = x, y, zstates_old
lml_min = optimize.minimize(log_marginal_likelihood.func_grad,
                            scaledparams,
                            args = args,
                            method='L-BFGS-B',
                            jac=True,
                            bounds=bounds,
                            options={'maxiter': 200000,
                                     'maxfun': 2000000,
                                     'ftol': 1e-9,
                                     'gtol': 1e-9,
                                     'disp': True})

Params here are: 0.005 0 0.1 0.1 0.5
Derivative of tau by brute force is -107.366389
Derivative of tau from equation is -107.365503
Params here are: 0.00497808 0 0.099922 0.073315 0.193256
Derivative of tau by brute force is -97.345153
Derivative of tau from equation is -97.345291
Params here are: 0.00488864 0 0.184949 0.0368938 0.000938695
Derivative of tau by brute force is 505.951337
Derivative of tau from equation is 506.012608
Params here are: 0.00496569 0 0.108748 0.0667104 0.092914
Derivative of tau by brute force is -98.316720
Derivative of tau from equation is -98.316856
Params here are: 0.00491746 0 0.15148 0.0460949 0.00528052
Derivative of tau by brute force is -80.806762
Derivative of tau from equation is -80.804843
Params here are: 0.00480566 0 0.354951 0.0513107 4.80146e-06
Derivative of tau by brute force is 23159809.439071
Derivative of tau from equation is 23162128.440669
Params here are: 0.00488302 0 0.1965 0.0476299 0.000621303
Derivative of tau by brute force is 12

In [10]:
lml_min

      fun: -120.81014713472621
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.60908595e-05,   1.45632164e-02,  -3.54305475e-06,
        -1.08769687e-04,   2.28525560e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 29
      nit: 22
   status: 0
  success: True
        x: array([-5.29328865,  0.        , -0.37591214, -2.49047032, -6.07015747])

In [11]:
res = hyperparameters.descale(lml_min.x)
res[4] = np.sqrt(1 / res[4])
print('\n'.join(['{:g}'.format(x) for x in list(res)]))

0.00500008
0
0.686663
0.082871
0.00231081
